# Stock Analysis Notebook

This Jupyter notebook is designed to analyze stocks based on various filters and sentiment analysis of news articles. 

## Dependencies Installation

First, we need to install the necessary dependencies. Uncomment the lines below if you haven't installed these packages yet.



In [11]:
'''
%pip install finvizfinance
%pip install pandas
%pip install transformers
%pip install yfinance
%pip install goose3
%pip install requests
%pip install ipywidgets
%pip install torch
%pip install tensorflow
%pip install nltk
import nltk
nltk.download('punkt')
'''

"\n%pip install finvizfinance\n%pip install pandas\n%pip install transformers\n%pip install yfinance\n%pip install goose3\n%pip install requests\n%pip install ipywidgets\n%pip install torch\n%pip install tensorflow\n%pip install nltk\nimport nltk\nnltk.download('punkt')\n"



## Import Libraries

Next, we import the necessary libraries for our analysis.



In [12]:
from finvizfinance.screener.overview import Overview # type: ignore
from finvizfinance.quote import finvizfinance        # type: ignore
from IPython.display import display                  # type: ignore
import pandas as pd                                  # type: ignore
from transformers import pipeline                    # type: ignore
import yfinance as yf                                # type: ignore
from goose3 import Goose                             # type: ignore
from requests import get                             # type: ignore
from nltk.tokenize import sent_tokenize              # type: ignore
from transformers import AutoTokenizer               # type: ignore
from bs4 import BeautifulSoup                        # type: ignore
import csv
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_colwidth', None) # Display full text in pandas dataframe / no line wrapping



## Stock Filtering

We define our filters for selecting stocks. These filters can be modified as per your requirements.



In [13]:
FILTERS_DICT = {
    'Performance': 'Today +10%',     # Day increase 10%
    'Relative Volume': 'Over 5',       # High Relative Volume
    'Price': 'Under $20',              # Price under 20 USD
    'Float': 'Under 10M'               # Float under 10 million
}

# Alternative filtering to consider:
'''
FILTERS_DICT = {'Debt/Equity':'Under 1',                 # Positive Operating Margin
                'PEG':'Low (<1)',                        # Debt-to-Equity ratio under 1
                'Operating Margin':'Positive (>0%)',     # Low P/B (under 1)
                'P/B':'Low (<1)',                        # Low P/E ratio (under 15)
                'P/E':'Low (<15)',                       # Low PEG ratio (under 1)
                'InsiderTransactions':'Positive (>0%)<'} # Positive Insider Transactions
'''


# The filters and general manual link for the finvizfinance library: https://finvizfinance.readthedocs.io/_/downloads/en/latest/pdf/ 
# Possible filters can be found by running the following code:

#from finvizfinance.screener.overview import Overview # type: ignore
#foverview = Overview()    # Create Overview object
#foverview.get_filters()   # Get list of all possible filters

# And after to see the possible options for a filter, run the following code:
#foverview.get_filter_options('Relative Volume') # Get list of all possible options for a filter, example on 'Relative Volume'


"\nFILTERS_DICT = {'Debt/Equity':'Under 1',                 # Positive Operating Margin\n                'PEG':'Low (<1)',                        # Debt-to-Equity ratio under 1\n                'Operating Margin':'Positive (>0%)',     # Low P/B (under 1)\n                'P/B':'Low (<1)',                        # Low P/E ratio (under 15)\n                'P/E':'Low (<15)',                       # Low PEG ratio (under 1)\n                'InsiderTransactions':'Positive (>0%)<'} # Positive Insider Transactions\n"



## Fetching Filtered Stocks

We define a function `get_filtered_stocks()` to fetch the stocks that meet our filter criteria.



In [14]:
def get_filtered_stocks():
    """
    Returns a list of tickers with:

    """
    
    foverview = Overview()
    foverview.set_filter(filters_dict=FILTERS_DICT)
    df_overview = foverview.screener_view()
    if not os.path.exists('out'): #ensures you have an 'out' folder ready
        os.makedirs('out')
    df_overview.to_csv('out/Overview.csv', index=False)
    
    tickers = df_overview['Ticker'].to_list()
    display(df_overview)
    return tickers



## Sentiment Analysis

We define a function `get_ticker_news_sentiment(ticker)` to perform sentiment analysis on the news articles of a given ticker.



In [15]:
def get_ticker_news_sentiment(ticker):
    """
    Returns a Pandas dataframe of the given ticker's most recent news article headlines,
    with the overal sentiment of each article.

    Args:
        ticker (string)

    Returns:
        pd.DataFrame: {'Date', 'Article title', Article sentiment'}
    """
    
    ALLOW_TOKENIZATION = False # True: the model will feed the article into the model in chunks of 512 tokens, 
    #                            False: the model will consider only the first sentences of the article until the total number of tokens does not exceed 512

    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

    ticker_news = yf.Ticker(ticker)
    try:
        news_list = ticker_news.get_news()
    except:
        print(f"Error getting news for ticker {ticker}")
        return
    
    extractor = Goose()
    pipe = pipeline("text-classification", model="ProsusAI/finbert")
    data = []
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
                
    for dic in news_list:
        title = dic['title']
        response = get(dic['link'], headers=headers)
        
        article = extractor.extract(raw_html=response.content)
        text = article.cleaned_text
        date = article.publish_date
        
        if date == None: # If the date is not found in the article, try to find it in the article's html
            soup = BeautifulSoup(response.text, 'html.parser')

            # Yahoo Finance usually stores the publication date in a 'time' tag with the class 'caas-attr-meta-time'
            date_tag = soup.find('time', {'class': 'caas-attr-meta-time'})
            if date_tag:
                date = date_tag['datetime']
            else:
                print('Publication date not found, article link for debugging', dic['link'])
            
            
        if len(text) > 512:
            if ALLOW_TOKENIZATION: # feed the article into the model in chunks of 512 tokens
                inputs = tokenizer.encode_plus(
                    text,
                    max_length=510,
                    truncation='longest_first',  # Truncate the longest sequences first
                    padding='max_length',  # Pad sequences to the max length
                    return_tensors='pt',  # Return PyTorch tensors
                )
                
                # Convert tensor to list and then to string
                input_ids = inputs["input_ids"].tolist()[0]
                new_text = tokenizer.decode(input_ids)
            
            else: # count the sentences until the total number of tokens does not exceed 512 (consider only first sentences of the article)
                # Split the text into sentences
                sentences = sent_tokenize(text)

                # Initialize an empty string for the new text
                new_text = ''

                # Add sentences to the new text until it exceeds 512 tokens
                for sentence in sentences:
                    if len(new_text.split()) + len(sentence) > 512:
                        new_text += ' ' + sentence
                    break

            # Now you can pass 'inputs' to your model
            results = pipe(new_text)

            data.append({'Ticker':f'{ticker}',
                         'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})

        else:
            results = pipe(text)
            data.append({'Ticker':f'{ticker}',
                         'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})
    df = pd.DataFrame(data)
    return df




## Generate CSV

We define a function `generate_csv(sentiment,ticker)` to generate a CSV file for each ticker's sentiment analysis.



In [16]:
def generate_csv(sentiment,ticker):
    sentiment.to_csv(f'out/{ticker}.csv', index=False)



## Fetch Sentiments and Generate CSVs

We fetch the sentiments for each undervalued stock and generate a CSV file for each.



In [17]:
undervalued = get_filtered_stocks()

sentiments = []
for ticker in undervalued:
    sentiment = get_ticker_news_sentiment(ticker)
    if sentiment is not None:
        sentiments.append(sentiment)
        generate_csv(sentiment,ticker)
    else:
        print(f'No news found for {ticker}')
    print(f'{ticker} News sentiment analysis done, {len(undervalued) - undervalued.index(ticker) - 1} stock tickers left')	


,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
0,DTCK,Davis Commodities Ltd.,Consumer Defensive,Farm Products,Singapore,34790000.0,8.64,1.42,0.1639,3682748.0
1,MULN,Mullen Automotive Inc,Consumer Cyclical,Auto Manufacturers,USA,44740000.0,NaN,6.83,0.1899,23377786.0
2,OXBR,Oxbridge Re Holdings Ltd,Financial,Insurance - Reinsurance,Cayman Islands,9260000.0,NaN,1.54,0.2126,300893.0
3,REBN,Reborn Coffee Inc,Consumer Cyclical,Restaurants,USA,7370000.0,NaN,3.19,0.8439,109736664.0
4,SGD,Safe and Green Development Corp,Real Estate,Real Estate - Development,USA,8900000.0,NaN,0.62,0.5881,69596215.0
5,SLNH,Soluna Holdings Inc,Financial,Capital Markets,USA,10220000.0,NaN,2.69,0.1070,13903909.0
6,SYTA,Siyata Mobile Inc,Technology,Communication Equipment,Canada,1490000.0,NaN,2.44,0.1529,627935.0
7,TIVC,Tivic Health Systems Inc,Healthcare,Medical Devices,USA,1970000.0,NaN,1.34,0.2762,23595550.0
8,WYY,Widepoint Corp,Technology,Information Technology Services,USA,24280000.0,NaN,2.54,0.1390,1201849.0


DTCK News sentiment analysis done, 8 stock tickers left
MULN News sentiment analysis done, 7 stock tickers left
OXBR News sentiment analysis done, 6 stock tickers left
REBN News sentiment analysis done, 5 stock tickers left
SGD News sentiment analysis done, 4 stock tickers left
SLNH News sentiment analysis done, 3 stock tickers left
SYTA News sentiment analysis done, 2 stock tickers left
TIVC News sentiment analysis done, 1 stock tickers left
WYY News sentiment analysis done, 0 stock tickers left




## Display Sentiments

Finally, we display the sentiments for each ticker.



In [18]:
for i in range(len(sentiments)):
    display(sentiments[i])

,Ticker,Date,Article title,Article sentiment
0,DTCK,2024-03-18T13:25:14.000Z,Is Davis Commodities Limited (NASDAQ:DTCK) A High Quality Stock To Own?,neutral


,Ticker,Date,Article title,Article sentiment
0,MULN,2024-05-07T13:30:00.000Z,Bollinger Motors Taps Amerit Fleet Solutions for Mobile Service & Warranty for the Company's B4 All-Electric Class 4 Commercial Truck,neutral
1,MULN,2024-05-07T13:00:00.000Z,Mullen Announces Bollinger Motors Taps Amerit Fleet Solutions for Mobile Service and Warranty for the Company’s B4 All-Electric Class 4 Commercial Truck,neutral
2,MULN,2024-05-06T14:02:16.000Z,Mullen Automotive Adds Commercial Dealer National Auto Fleet Group,neutral
3,MULN,2024-05-06T13:15:00.000Z,"Mullen Signs Agreement with California Based and HVIP Approved, National Auto Fleet Group",neutral
4,MULN,2024-05-06T13:00:00.000Z,Mullen Announces Adoption of Limited Duration Stockholder Rights Plan,neutral
5,MULN,2024-05-02T13:00:00.000Z,Pritchard EV and Mobile Road Service Solutions Debut All Electric Mullen ONE Roadside Assistance Vehicle at American Towman Showcase in Las Vegas,neutral
6,MULN,2024-05-01T13:00:00.000Z,"Mullen Adds One of the Largest US Commercial Dealers, Pritchard EV, to Dealer Network",positive
7,MULN,2024-04-30T12:13:36.000Z,"Exchange-Traded Funds, Equity Futures Lower Pre-Bell Tuesday Ahead of FOMC Meeting",neutral


,Ticker,Date,Article title,Article sentiment
0,OXBR,2024-05-01T20:05:00.000Z,"Oxbridge Re Announces 2024 First Quarter Results on May 9, 2024",neutral
1,OXBR,2024-04-08T12:30:00.000Z,Oxbridge Re to Present at the LD Micro Main Event XIV in New York,neutral
2,OXBR,2024-03-27T16:04:41.000Z,Oxbridge Re Holdings Limited (NASDAQ:OXBR) Q4 2023 Earnings Call Transcript,neutral
3,OXBR,2024-03-27T13:15:00.000Z,Oxbridge Re Announces Launch of Tokenized Real-World Asset Offering EpilsonCat Re,neutral
4,OXBR,2024-03-26T20:01:00.000Z,Oxbridge Re Holdings Limited Reports Fiscal 2023 Results,neutral
5,OXBR,2024-03-07T21:05:00.000Z,"Oxbridge Re Announces 2023 Fourth Quarter Results on March 26, 2024",neutral


,Ticker,Date,Article title,Article sentiment
0,REBN,2024-05-08T12:31:00.000Z,"Reborn Coffee Closes Master License Agreement for Guangdong Province, China",neutral
1,REBN,2024-03-28T12:31:00.000Z,Reborn Coffee Reports Fiscal Year 2023 Financial Results,neutral
2,REBN,2024-02-29T21:30:00.000Z,Reborn Coffee Closes $1.0 Million Private Placement Equity Investment from Accredited Investor,neutral
3,REBN,2024-02-13T13:31:00.000Z,Reborn Coffee Partners with Pet Fashion Brand Sgt. Puppa to Offer Pet Attire and Accessories at Pet Social Cafe,neutral


,Ticker,Date,Article title,Article sentiment
0,SGD,2024-05-07T13:15:00.000Z,"Safe and Green Development Corporation to Acquire MyVonia, a Cutting-Edge AI Asset",neutral
1,SGD,2024-04-30T13:00:00.000Z,Safe and Green Development Corporation Expects to Close the Sale of St. Mary’s Site in May 2024,neutral
2,SGD,2024-04-26T13:05:00.000Z,Safe and Green Development Corporation Executes Contract to Sell Lago Vista Site,neutral
3,SGD,2024-04-23T13:00:00.000Z,Safe and Green Development Corp Launches XENE Home AI Platform,neutral
4,SGD,2024-04-17T13:00:00.000Z,Safe and Green Development Corp Provides Quarterly Business Update,neutral
5,SGD,2024-04-04T13:00:00.000Z,Safe and Green Development Corp Announces XENE Home AI Beta Launch,neutral
6,SGD,2024-03-28T13:00:00.000Z,Safe and Green Development Corp Reports Record Fiscal Year 2023 Financial Results and Provides a Business Update,neutral
7,SGD,2024-03-19T12:30:00.000Z,"Safe & Green Holdings and Soldier on Sign Letter of Intent to Design, Build and Operate 800-Unit Supportive Housing Community for Veterans",neutral


,Ticker,Date,Article title,Article sentiment
0,SLNH,2024-05-08T12:00:00.000Z,Soluna Launches New AI Cloud Service in Collaboration with Leading High Performance Computing Company,neutral
1,SLNH,2024-05-06T12:00:00.000Z,Soluna Announces Monthly Business Update,neutral
2,SLNH,2024-04-22T12:00:00.000Z,Soluna to Participate in Water Tower Research Fireside Chat Series,neutral
3,SLNH,2024-04-12T14:07:00.000Z,AI & Technology Hybrid Investor Conference Agenda: Presentations Now Available for Online Viewing,neutral
4,SLNH,2024-04-09T14:26:00.000Z,AI & Technology Hybrid Investor Conference Presented by Water Tower Research Agenda Announced for April 10 and 11,neutral
5,SLNH,2024-04-03T12:00:00.000Z,Soluna Appoints John Tunison as Chief Financial Officer,neutral
6,SLNH,2024-04-03T11:51:35.000Z,Soluna Holdings Full Year 2023 Earnings: US$0.022 loss per share (vs US$185 loss in FY 2022),neutral
7,SLNH,2024-04-02T12:00:00.000Z,Soluna Reports Fourth Quarter and Full Year 2023 Results,neutral


,Ticker,Date,Article title,Article sentiment
0,SYTA,2024-05-08T13:00:00.000Z,Siyata Mobile Announces Pricing of its $4.0 Million Public Offering of Common Shares and Pre-Funded Warrants,neutral
1,SYTA,2024-04-08T20:05:00.000Z,Siyata Mobile Announces Fourth Quarter and Full-Year 2023 Financial Results,neutral
2,SYTA,2024-04-06T13:28:32.000Z,Siyata Mobile Full Year 2023 Earnings: Revenues Miss Expectations,neutral
3,SYTA,2024-04-05T12:00:00.000Z,"Siyata Mobile to Host Fourth Quarter and Year End 2023 Financial Results Call at 8:30 a.m. ET on April 9, 2024",neutral
4,SYTA,2024-04-04T12:00:00.000Z,"Siyata Adds New Channel Partner with 3AM Innovations, a Software Provider for First Responders",neutral
5,SYTA,2024-03-21T12:00:00.000Z,"Siyata Further Expands Global Distribution in Agreement with Leading Mobility, Logistics and Infrastructure Conglomerate in the Middle East",positive
6,SYTA,2024-03-11T15:43:00.000Z,Siyata Mobile Inc. to Showcase Portfolio at IWCE 2024,neutral
7,SYTA,2024-03-05T17:42:00.000Z,Siyata Mobile SD7 Push-to-Talk Over Cellular Device Featured,neutral


,Ticker,Date,Article title,Article sentiment
0,TIVC,2024-05-08T13:00:00.000Z,"Tivic Health Announces Successful Completion of Non-Invasive Vagus Nerve Stimulation Study (VNS); Demonstrates Clinically Effective Biological Changes in the Autonomic, Cardiac, and Central Nervous Systems",neutral
1,TIVC,2024-03-28T14:03:28.000Z,"Tivic Health Systems, Inc. (NASDAQ:TIVC) Q4 2023 Earnings Call Transcript",neutral
2,TIVC,2024-03-27T10:49:56.000Z,Tivic Health Systems Full Year 2023 Earnings: US$10.40 loss per share (vs US$104 loss in FY 2022),neutral
3,TIVC,2024-03-25T21:25:00.000Z,Tivic Reports Fiscal Year 2023 Financial Results,neutral
4,TIVC,2024-03-19T20:30:00.000Z,Tivic Health to Report 2023 Year-End Financial Results on March 25th Via Conference Call and Webcast,neutral
5,TIVC,2024-02-21T14:00:00.000Z,Tivic Health Announces Completion of Enrollment for Study of Novel Non-Invasive Bioelectronic Approach to Vagus Nerve Stimulation,neutral


,Ticker,Date,Article title,Article sentiment
0,WYY,2024-05-08T13:00:00.000Z,"WidePoint Selected by the U.S. Navy for the 10-Year, $2.7 Billion Spiral 4 Contract",neutral
1,WYY,2024-05-01T13:00:00.000Z,"WidePoint Sets First Quarter 2024 Conference Call for Wednesday, May 15, 2024 at 4:30 p.m. ET",neutral
2,WYY,2024-04-23T13:00:00.000Z,WidePoint Awarded $22.7 Million in Tech and Cyber Security Services Contracts During Q1 2024,neutral
3,WYY,2024-04-09T12:23:00.000Z,Looking for the Next Google? Buy These 7 Tech Penny Stocks,neutral
4,WYY,2024-03-27T16:07:31.000Z,WidePoint Corporation (AMEX:WYY) Q4 2023 Earnings Call Transcript,neutral
5,WYY,2024-03-27T10:17:53.000Z,Q4 2023 WidePoint Corp Earnings Call,neutral
6,WYY,2024-03-26T21:11:43.000Z,WidePoint: Q4 Earnings Snapshot,neutral
7,WYY,2024-03-26T20:05:00.000Z,WidePoint Reports Fourth Quarter and Full Year 2023 Financial Results,neutral




## Additional Code

The following code snippets are not integrated into the main flow of the notebook but can be used for additional analysis.



In [19]:
'''
quote = finvizfinance('SGE')
df = quote.ticker_inside_trader()
from datetime import datetime
today = datetime.today().date()
df = quote.ticker_news()
df = df[df['Date'].dt.date == today]
df
df = quote.ticker_fundament()
df
'''

"\nquote = finvizfinance('SGE')\ndf = quote.ticker_inside_trader()\nfrom datetime import datetime\ntoday = datetime.today().date()\ndf = quote.ticker_news()\ndf = df[df['Date'].dt.date == today]\ndf\ndf = quote.ticker_fundament()\ndf\n"